In [30]:
import pandas as pd
# Preprocessing
import re
import preprocessor as p
import string
import contractions
from collections import defaultdict
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.corpus import stopwords

In [31]:
# Download requires corpus
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<h2> Consolidation </h2>

In [32]:
%%script false
# Consolidate all csvs into one dataframe then export as csv
fb_df = pd.read_csv('./data/social_media/Facebook_Consolidated.csv')
fb_df["platform"] = "Facebook"
reddit_df = pd.read_csv('./data/social_media/reddit.csv')
reddit_df["platform"] = "Reddit"
tiktok_df = pd.read_excel('./data/social_media/Tiktok.xlsx')
tiktok_df["platform"] = "Tiktok"
youtube_df = pd.read_csv('./data/social_media/Youtube.csv')
youtube_df["platform"] = "Youtube"
# text column
# fb_df["0"]
# reddit_df["body"]
# tiktok_df["Comment Text"]
# youtube_df["Comment"]
# Create master_series
text = fb_df["0"].append(reddit_df["body"], ignore_index=True).append(tiktok_df["Comment Text"], ignore_index=True).append(youtube_df["Comment"], ignore_index=True)
platform = fb_df["platform"].append(reddit_df["platform"], ignore_index=True).append(tiktok_df["platform"], ignore_index=True).append(youtube_df["platform"], ignore_index=True)
# Create master_df
master_df = pd.DataFrame({"text": text, "platform": platform})
# Drop master_df NaN
master_df = master_df[master_df['text'].notna()]
master_df.to_csv('./data/consolidated.csv', index=False)


Couldn't find program: 'false'


<h2> Preprocessing </h2>
TODO: Remove entries that have only 1 word?

In [33]:
dataset = pd.read_csv('./data/consolidated.csv')
dataset=dataset.rename(columns = {'text':'raw'})
dataset = dataset[["platform", "raw"]]
# dataset = pd.read_csv('./data/sample.csv')
# dataset.columns = ['raw']

In [34]:
# Create new column for prepr preprocessed
dataset["preprocessed"] = dataset["raw"]

In [35]:
dataset.head()

,platform,raw,preprocessed
0,Facebook,Why nowadays every thing seem to be increasin...,Why nowadays every thing seem to be increasin...
1,Facebook,I will have to disagree.. we’re not that high!!,I will have to disagree.. we’re not that high!!
2,Facebook,"Wag po tayong mag-alala. Naniniwala po ako, is...","Wag po tayong mag-alala. Naniniwala po ako, is..."
3,Facebook,Ok lang yang lahat naman nang bansa ganyan. Sa...,Ok lang yang lahat naman nang bansa ganyan. Sa...
4,Facebook,Sama-sama tayong BABAON muli.,Sama-sama tayong BABAON muli.


In [36]:
lemma = WordNetLemmatizer()

# stop words
stop = set(stopwords.words('english'))
# include filipino stopwords
f = open("./data/stopwords.txt", "r")
for line in f:
    stop.add(line.strip())
f.close()

# Helper functions
def remove_stopwords(text):
    words = [w for w in text if w not in stop]
    return words

def lemmatize_text(tokenized_text):
    return [lemma.lemmatize(w) for w in tokenized_text]

In [37]:
# Convert to lowercase
dataset["preprocessed"] = [entry.lower() for entry in dataset["preprocessed"]]
# Remove punctuation marks
dataset["preprocessed"] = [entry.translate(str.maketrans('', '', string.punctuation)) for entry in dataset["preprocessed"]]
# Twitter preprocess
dataset["preprocessed"] = [p.clean(entry) for entry in dataset["preprocessed"]]
# Tokenization
dataset['preprocessed']= [word_tokenize(entry) for entry in dataset['preprocessed']]
# Normalizatinon
dataset['preprocessed'] = dataset['preprocessed'].apply(lemmatize_text)
# Remove stopwords
dataset["preprocessed"] = [remove_stopwords(entry) for entry in dataset["preprocessed"]]
# # Turn to String
# dataset['preprocessed']= [str(entry) for entry in dataset['preprocessed']]

In [41]:
type(dataset["preprocessed"][0])

list

In [38]:
dataset.head()

,platform,raw,preprocessed
0,Facebook,Why nowadays every thing seem to be increasin...,"[nowadays, every, thing, seem, increasing, gov..."
1,Facebook,I will have to disagree.. we’re not that high!!,"[disagree, high]"
2,Facebook,"Wag po tayong mag-alala. Naniniwala po ako, is...","[wag, po, tayong, magalala, naniniwala, po, is..."
3,Facebook,Ok lang yang lahat naman nang bansa ganyan. Sa...,"[ok, lang, yang, naman, nang, bansa, ganyan, s..."
4,Facebook,Sama-sama tayong BABAON muli.,"[samasama, tayong, babaon]"


In [39]:
dataset.to_csv('./data/preprocessed.csv', index=False)

In [40]:
# preprocessed = pd.read_csv('./data/preprocessed.csv')
# df = pd.DataFrame({"raw":dataset["text"], "preprocessed":preprocessed["text"]})